# **Lab-11-2 RNN hihello and charseq**

## **'Hihello' problem**


*   문제 : hihello라는 문자열을 예측하는 모델을 만드는 것
*   목적 : 다음에 오는 character를 예측해야 함 (h가 들어오면 i, i가 들어오면 h를 예측해야 함)

## **One-hot Encoding**

Characters를 컴퓨터에서 표현하는 방식을 Embedding이라 하고 대표적으로는 One-hot Encoding이 있다.

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2Fb5Kwpc%2FbtqC2s6tKcO%2FAImC1hFc3ryIcyWN0kRmA0%2Fimg.png)



*   각 단어에추가 index를 부여하는 방식 
*   h의 경우 [1, 0, 0, 0, 0] 으로 자기 index의 값만 1을 가지는 방식 

## **Cross Entropy Loss**
*   Categorical Output을 예측하는데 사용하는 loss 

    cirterion = torch.nn.CrossEntropyLoss()  
    criterion(모델의 Output, 실제 값) 

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2FcE1uEN%2FbtqC3OA2y0B%2FZA3SHC7Be7KmEtsGkFOVk0%2Fimg.png)










## **Code Run - 'hihello'**

In [0]:
import torch
import torch.optim as optim
import numpy as np

In [2]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

In [0]:
#declare dictionary
char_set = ['h', 'i', 'e', 'l', 'o']

In [0]:
# hyper parameters
input_size = len(char_set)
hidden_size = len(char_set) # 편의상 input과 맞춤
learning_rate = 0.1

*  hidden_size는 임의로 설정 




In [0]:
# data setting
x_data = [[0, 1, 0, 2, 3, 3]] # hihell
x_one_hot = [[[1, 0, 0, 0, 0],
              [0, 1, 0, 0, 0],
              [1, 0, 0, 0, 0],
              [0, 0, 1, 0, 0],
              [0, 0, 0, 1, 0],
              [0, 0, 0, 1, 0]]]
y_data = [[1, 0, 2, 3, 3, 4]] # ihello

In [0]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)



*   torch.FloatTensor로 one_hot vector를 Floattensor형태로 변환 



In [0]:
# declare RNN
rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True) 
# batch_first guarantess the order of output = (B, S, F)

In [0]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

In [10]:
# start training
for i in range(100) :
  optimizer.zero_grad()
  outputs, _status = rnn(X)
  loss = criterion(outputs.view(-1, input_size), Y.view(-1))
  loss.backward()
  optimizer.step()

  result = outputs.data.numpy().argmax(axis=2)
  result_str = ''.join([char_set[c] for c in np.squeeze(result)])
  print(i, "loss: ", loss.item(), "prediction", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  1.7802648544311523 prediction [[1 1 1 1 1 1]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  iiiiii
1 loss:  1.4931949377059937 prediction [[1 4 1 1 4 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ioiioo
2 loss:  1.33371102809906 prediction [[1 3 2 3 1 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ilelio
3 loss:  1.215294599533081 prediction [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  elelll
4 loss:  1.1131387948989868 prediction [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  elelll
5 loss:  1.0241864919662476 prediction [[2 3 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  elello
6 loss:  0.9573140740394592 prediction [[2 3 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  elello
7 loss:  0.910200297832489 prediction [[2 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ehello
8 loss:  0.8731765151023865 prediction [[1 0 2 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]] prediction str:  ihello
9 loss:  0.839961767196

## **Code Run - charseq**

In [26]:
# Random seed to make results deterministic and reproducible
torch.manual_seed(0)

In [0]:
sample = " if you want you"

In [28]:
# make dictionary
char_set = list(set(sample))
char_dic = {c: i for i, c in enumerate(char_set)}
print(char_dic)

{' ': 0, 'i': 1, 'w': 2, 'f': 3, 'u': 4, 'a': 5, 'y': 6, 't': 7, 'o': 8, 'n': 9}


**집합 자료형 set**

1. 리스트입력

  set([1,2,3]) -> {1, 2, 3}

2. 문자열 입력

  set("Hello") -> {'e', 'H', 'l', 'o'}

**특징**
*   중복을 허용하지 않는다.
*   순서가 없다(Unordered).




In [0]:
# hyper parameters
dic_size = len(char_dic)
hidden_size = len(char_dic)
learning_rate = 0.1

In [0]:
# data setting
sample_idx = [char_dic[c] for c in sample]
x_data = [sample_idx[:-1]] # 맨 뒤에 하나 빼고
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
y_data = [sample_idx[1:]] # 맨 앞에 하나 빼고



*   이전과 다르게 dictionary형태로 저장 
*   np.eye : 대각성분은 '1'이고 나머지 성분은 '0'으로 구성된 정방행렬인 항등행렬(identity matrix) 혹은 **단위행렬**(unit matrix)을 만듬 



In [0]:
# transform as torch tensor variable
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [0]:
# declare RNN
rnn = torch.nn.RNN(dic_size, hidden_size, batch_first=True)

In [0]:
# loss & optimizer setting
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn.parameters(), learning_rate)

In [34]:
# start training
for i in range(50):
    optimizer.zero_grad()
    # _status : 만약 다음 input이 있으면 그걸 RNN안에서 계산할 때 쓰이게 될 hidden state
    # 여기서는 주어진 모든 input을 다 처리하고 나오기 때문에 따로 쓰이지는 않는다.
    outputs, _status = rnn(X)
    # view를 쓰는 이유는 shape을 batch_dimension이 앞에 오도록 바꿔주기 위함이다.
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    # index=2인 dim, 즉 어떤 character인지 나타내는 dim에서 가장 큰 index만 가져온다.
    result = outputs.data.numpy().argmax(axis=2)
    # join을 통해서 하나의 string으로.
    # np.sqeeze는 shape에서 dim=1인 축을 없애준다.
    result_str = ''.join([char_set[c] for c in np.squeeze(result)])
    print(i, "loss: ", loss.item(), "prediction: ", result, "true Y: ", y_data, "prediction str: ", result_str)

0 loss:  2.2919626235961914 prediction:  [[5 8 7 5 8 8 7 5 8 7 0 0 5 7 8]] true Y:  [[1, 3, 0, 6, 8, 4, 0, 2, 5, 9, 7, 0, 6, 8, 4]] prediction str:  aotaootaot  ato
1 loss:  1.9778224229812622 prediction:  [[5 0 7 6 8 0 0 0 5 0 0 0 6 8 0]] true Y:  [[1, 3, 0, 6, 8, 4, 0, 2, 5, 9, 7, 0, 6, 8, 4]] prediction str:  a tyo   a   yo 
2 loss:  1.7548198699951172 prediction:  [[4 0 0 6 8 0 0 0 4 0 0 0 0 8 0]] true Y:  [[1, 3, 0, 6, 8, 4, 0, 2, 5, 9, 7, 0, 6, 8, 4]] prediction str:  u  yo   u    o 
3 loss:  1.5657832622528076 prediction:  [[1 8 0 6 8 4 0 2 5 0 0 0 6 8 4]] true Y:  [[1, 3, 0, 6, 8, 4, 0, 2, 5, 9, 7, 0, 6, 8, 4]] prediction str:  io you wa   you
4 loss:  1.4374821186065674 prediction:  [[1 8 0 6 8 4 0 2 5 9 0 0 6 8 4]] true Y:  [[1, 3, 0, 6, 8, 4, 0, 2, 5, 9, 7, 0, 6, 8, 4]] prediction str:  io you wan  you
5 loss:  1.3541741371154785 prediction:  [[1 3 0 6 8 4 5 2 5 9 7 0 6 8 4]] true Y:  [[1, 3, 0, 6, 8, 4, 0, 2, 5, 9, 7, 0, 6, 8, 4]] prediction str:  if youawant you
6 loss:  1